# SlidesGPT


In [1]:
# Imports
import re
import os

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage


/Users/samarthgoel/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")


## Get concept design


# Determine which slide designs to use


# Input: Topic; Output: Outline


In [15]:
chat = ChatOpenAI(temperature=0.4, openai_api_key=openai_api_key)


In [30]:
topic = "benefits of generative AI to reducing cloud computing spend"
goal = "sell the benefits of generative AI for enterprise use cases"
audience = "CIO of a big multinational corporation"

# use some adjectives to describe how you want the presentation to feel
adjectives = ["adventurous", "impressive", "revolutionary"]


In [126]:
template = """
Can you create an outline for a slide-based presentation given a topic, \
and have each section have at least 2 sub-sections. Use integers for section titles, \
letters for sub-section titles, and roman numerals for sub-sub-section titles. \
Make sure to include a section for a concluding slide. \
Do not output any text besides the outline.

Example:
I. ...
    A. ...
    B. ...
        1. ...
        2. ...
        3. ...
...
V. ...
    A. ...
    B. ...
    C. ...

Topic: {topic}
Outline:
"""

prompt_template = PromptTemplate(template=template, input_variables=["topic"])

outline_prompt = prompt_template.format(topic=topic)


In [127]:
out = chat(
    [
        SystemMessage(
            content="You are a helpful ai bot that creates presentations based on user-given topic, inputs, and data"
        ),
        HumanMessage(content="Hi! Can you help me create a powerpoint presentation?"),
        AIMessage(
            content="Of course! What do you want to make a powerpoint presentation about, what do you want its goal to be, and who is the target audience?"
        ),
        HumanMessage(
            content=f"I am making a presentation about the {topic}. I want to {goal}. My audience is the {audience}"
        ),
        AIMessage(
            content="Thank you for the information! What are 3 adjectives you would use to describe the design and feel of the presentation?"
        ),
        HumanMessage(content=", ".join(adjectives)),
        AIMessage(
            content="Thank you! How would you like the presentation to be formatted?"
        ),
        HumanMessage(content=outline_prompt),
    ]
)

In [128]:
print(out.content)


I. Introduction
    A. Definition of generative AI
    B. Importance of reducing cloud computing spend
II. Benefits of generative AI
    A. Cost savings
        1. Reduction in hardware and software costs
        2. Decrease in energy consumption
    B. Increased efficiency
        1. Faster data processing
        2. Improved resource allocation
III. Use cases for generative AI in reducing cloud computing spend
    A. Predictive maintenance
        1. Early detection of hardware failures
        2. Optimization of maintenance schedules
    B. Resource allocation
        1. Real-time allocation of computing resources
        2. Optimization of resource usage
IV. Challenges and limitations of using generative AI for reducing cloud computing spend
    A. Data privacy and security concerns
        1. Protection of sensitive data
        2. Compliance with regulations
    B. Complexity of implementation
        1. Integration with existing systems
        2. Training and deployment of AI m

In [129]:
response = out.content
outline = ""

if "\t" in response:
    outline = response
else:
    response = response.split("\n")

    for i in range(len(response)):
        if any(char in response[i] for char in ["A.", "B.", "C.", "D.", "E."]):
            response[i] = "\t" + response[i]
        if any(char in response[i] for char in ["1.", "2.", "3.", "4.", "5."]):
            response[i] = "\t\t" + response[i]

    outline = "\n".join(response)

In [130]:
slide_type_template = f"""
Can you use the following outline to create a presentation on the {topic} for a {audience}. \

For each slide, specify which slide type to use.
Choose from the following slide types: [Title Slide, Title and Body, Section Header, Title and Two Columns, \
Title and One Column, Main Point, Section Title and Description, Image and Caption, Big Number]
For example, section titles should use a Section Header slide type.

For the slide content, try to use a mix of 30% images and 70% text. \
Image slides should describe what they are trying to convey with the image. \
Text slides should mostly be bullet points, but with some sentences.
Also create a title slide for the presentation that uses a Title Slide slide type. \
This should be the only slide that uses the Title Slide slide type.

Format each slide like this:
<slide title>: <slide type>
<slide content>

Outline:
{outline}

Slides:
"""

In [131]:
out = chat(
    [
        SystemMessage(
            content="You are a helpful ai bot that creates presentations based on user-given topic, inputs, and data"
        ),
        HumanMessage(content="Hi! Can you help me create a powerpoint presentation?"),
        AIMessage(
            content="Of course! What do you want to make a powerpoint presentation about, what do you want its goal to be, and who is the target audience?"
        ),
        HumanMessage(
            content=f"I am making a presentation about the {topic}. I want to {goal}. My audience is the {audience}"
        ),
        AIMessage(
            content="Thank you for the information! What are 3 adjectives you would use to describe the design and feel of the presentation?"
        ),
        HumanMessage(content=", ".join(adjectives)),
        AIMessage(
            content="Thank you! How would you like the presentation to be formatted?"
        ),
        HumanMessage(content=outline_prompt),
        AIMessage(content=outline),
        HumanMessage(content=slide_type_template),
    ]
)

In [132]:
print(out.content)


Title Slide: Title Slide
Title: Benefits of Generative AI to Reducing Cloud Computing Spend

Slide 1: Section Header
Title: Introduction
Subtitle: Definition of generative AI and importance of reducing cloud computing spend

Slide 2: Title and Body
Title: Definition of Generative AI
Body:
- Generative AI is a subset of artificial intelligence that involves using algorithms to generate new content
- Examples include image and text generation, music composition, and video creation

Slide 3: Title and Body
Title: Importance of Reducing Cloud Computing Spend
Body:
- Cloud computing can be a significant expense for businesses
- Reducing cloud computing spend can free up resources for other areas of the business
- Generative AI can help reduce cloud computing spend while improving efficiency

Slide 4: Section Header
Title: Benefits of Generative AI

Slide 5: Title and Two Columns
Title: Cost Savings
Left Column:
- Reduction in hardware and software costs
- Decrease in energy consumption
Righ

In [140]:
adjustment_template = """
Can you add slightly more detail to the body of each slide? \
Make each bullet point less general and make sure each slide has at least 3 bullet points on it. \
Add more slides to the presentation if needed to add sufficient detail. \
Use proper grammar for each bullet point. \
For each image, enclose the image description in brackets. \
Do not use more than one image per slide.
"""


In [141]:
out = chat(
    [
        SystemMessage(
            content="You are a helpful ai bot that creates presentations based on user-given topic, inputs, and data"
        ),
        HumanMessage(content="Hi! Can you help me create a powerpoint presentation?"),
        AIMessage(
            content="Of course! What do you want to make a powerpoint presentation about, what do you want its goal to be, and who is the target audience?"
        ),
        HumanMessage(
            content=f"I am making a presentation about the {topic}. I want to {goal}. My audience is the {audience}"
        ),
        AIMessage(
            content="Thank you for the information! What are 3 adjectives you would use to describe the design and feel of the presentation?"
        ),
        HumanMessage(content=", ".join(adjectives)),
        AIMessage(
            content="Thank you! How would you like the presentation to be formatted?"
        ),
        HumanMessage(content=outline_prompt),
        AIMessage(content=outline),
        HumanMessage(content=slide_type_template),
        AIMessage(content=out.content),
        HumanMessage(content=adjustment_template),
    ]
)

In [142]:
print(out.content)


Title Slide: Title Slide
Title: Benefits of Generative AI to Reducing Cloud Computing Spend

Slide 1: Section Header
Title: Introduction
Subtitle: Definition of generative AI and importance of reducing cloud computing spend

Slide 2: Title and Body
Title: Definition of Generative AI
Body:
- Generative AI refers to the use of algorithms to create new content, such as images, music, or text, without human intervention [Example of generative AI-generated image of a landscape with a caption describing it]
- It can also refer to the use of AI to create new solutions to problems, such as optimizing resource allocation or predicting hardware failures [Example of generative AI-generated solution for resource allocation with a caption describing it]
- Generative AI is a powerful tool that can help businesses reduce costs and improve efficiency by automating tasks and creating new solutions to problems.

Slide 3: Title and Body
Title: Importance of Reducing Cloud Computing Spend
Body:
- Cloud co